In [1]:
%load_ext lab_black

In [2]:
from liltab.data.datasets import PandasDataset
from liltab.data.dataloaders import FewShotDataLoader, ComposedDataLoader
from liltab.model.heterogenous_attributes_network import HeterogenousAttributesNetwork
from pathlib import Path

In [3]:
paths = [
    Path("data/openml_original/ICU.csv"),
    Path("data/openml_original/autoPrice.csv"),
    Path("data/openml_original/fri_c3_250_10.csv"),
]

pd_datasets = [PandasDataset(path) for path in paths]
few_shot_datasets = [
    FewShotDataLoader(pd_dataset, 5, 27, 100) for pd_dataset in pd_datasets
]
composed_few_shot_dataset = ComposedDataLoader(
    dataloaders=few_shot_datasets, n_episodes=100
)

In [4]:
network = HeterogenousAttributesNetwork()

X_support, y_support, X_query, y_query = next(composed_few_shot_dataset)
print(X_support.shape)
network(X_support, y_support, X_query).shape

torch.Size([5, 19])


torch.Size([27, 1])